### Isobaric fractional crystallisation
New operational ideas in here:
- Playing around somewhat differently with construction of PT_steps and the list of possible phases
- Reading in the xyzguess block from a text file. This format will typically be simpler for those coming from the THERMOCALC end. 
- THERMOCALC runs are veeeeery long here. We save the results out to disk here to allow job resumption. Note that the scripts themselves can also be loaded/saved, and it's up to the user to orchestrate saving/load/restarting of their simulations.   

In [1]:
import os
import tawnycalc as tc

rerun_all = False  # rerun results even if previous calculations available on disk.

In [2]:
# define constant pressure and range/interval in temperature
Pfrac = 4.0
Tstart = 1100
Tend = 700
Tint = 100

PT_steps = []
[ PT_steps.append( (Pfrac,t) ) for t in range(Tend,Tstart,Tint) ]
PT_steps.reverse()
PT_steps

[(4.0, 1000), (4.0, 900), (4.0, 800), (4.0, 700)]

In [3]:
# Define phases to be considered in G-minimisation

# all phases as single string
phasestr = "cpx opx spn g plc ksp ilm bi q fl liq"

# crystalline phases (to be removed) as list
crystphases = list(set(phasestr.split()) - set( ["liq","fl"] ))   

In [4]:
# To start with a clean context, set `scripts_dir=None`
context = tc.Context(scripts_dir=None)
context.prefs

calcmode   : 1
scriptfile : mcnpkn
dataset    : None

In [5]:
# Set the required dataset on the `prefs` dictionary.
context.prefs["dataset"] = 633

In [6]:
# Additional scripts

context.script[       "axfile"] = "ig50NCKFMASHTOCr"
context.script[         "with"] = phasestr  # replaces 'which'
context.script[       "dogmin"] = "1"                 # replaces 'dogmin yes 1'
context.script[       "maxvar"] = "12"                 # replaces 'setmaxvar'
context.script[    "diagramPT"] = "2 20 400 1100"     # new script
context.script["pseudosection"] = ""                  # replaces 'pseudosection yes'
context.script[   "samecoding"] = ["spn cm"]  # 

In [7]:
# Define the initial bulk composition. 
# This is done in 'rbi' form, i.e. by setting the modes and compositions
# of the individual phases, to be summed by THERMOCALC:
#  rbi_sec.add_phase("<phase 1>", "<mode>", "<oxide values>")
#  rbi_sec.add_phase("<phase 1>", "<mode>", "<oxide values>")
#    ....
# Alternatively, the user can set up the bulk composition as a single string:
#  rbi_sec.add_phase("blah", "1", "<oxide values>")

rbi_sec = tc.rbi(oxides="  H2O SiO2 Al2O3 CaO MgO FeO K2O Na2O TiO2 O Cr2O3")
rbi_sec.add_phase(  "liq",  "1"     , "0.8  81.93   8.65  1.16   0.16  0.57   3.19  4.21   0.12   0.04   0.01")
context.script["rbi"] = rbi_sec

In [8]:
# Create xyzguess section, reading from local file
# (easier route for people coming from the thermocalc end!)

# First create an empty thermocalc_script object. These objects
# will handle reading of data from tc script files.
xyzguess_script = tc.thermocalc_script()
xyzfile=os.path.join(os.getcwd(), "ecrg-frac-xtn-xyzguess.txt")
# Load xyzguess data now
xyzguess_script.load(xyzfile)
# Check
xyzguess_script

xyzguess :
    wo(L)   : 0.107216  
    sl(L)   : 0.0873506 
    fo(L)   : 0.0142023 
    fa(L)   : 0.0109878 
    jd(L)   : 0.151907  
    hm(L)   : 0.00154624
    ek(L)   : 9.81910e-6
    ti(L)   : 0.00413534
    kj(L)   : 0.0368694 
    yct(L)  : 0.0885231 
    h2o(L)  : 0.219727  
    wo(fl)  : 0.000103865
    sl(fl)  : 5.87807e-5
    fo(fl)  : 1.41812e-8
    fa(fl)  : 3.11791e-8
    jd(fl)  : 0.000862050
    hm(fl)  : 0.000159068
    ek(fl)  : 3.29610e-7
    ti(fl)  : 0.000162396
    kj(fl)  : 0.00562408
    h2o(fl) : 0.985936  
    ca(plc) : 0.6       
    k(plc)  : 0.0116338 
    x(ol)   : 0.102434  
    c(ol)   : 0.00275804
    Q(ol)   : 0.000156353
    x(opx)  : 0.0956777 
    y(opx)  : 0.182193   range      0.000      2.000     
    c(opx)  : 0.0489271 
    Q(opx)  : -0.0475598 range      -1.000     1.000     
    f(opx)  : 0.0177886 
    t(opx)  : 0.00823030
    cr(opx) : 0.0205952 
    j(opx)  : 0.00906326
    x(cpx)  : 0.0964208 
    y(cpx)  : 0.0825558  range      0.000  

In [9]:
# Now set data onto the context's script
context.script["xyzguess"] = xyzguess_script["xyzguess"]

In [10]:
context.script

axfile        : ig50NCKFMASHTOCr
autoexit      : yes
with          : cpx opx spn g plc ksp ilm bi q fl liq
dogmin        : 1
maxvar        : 12
diagramPT     : 2 20 400 1100
pseudosection : 
samecoding :
    spn cm

rbi :
          H2O  SiO2   Al2O3  CaO   MgO   FeO   K2O   Na2O  TiO2  O     Cr2O3
liq  1.0  0.8  81.93  8.65   1.16  0.16  0.57  3.19  4.21  0.12  0.04  0.01

xyzguess :
    wo(L)   : 0.107216  
    sl(L)   : 0.0873506 
    fo(L)   : 0.0142023 
    fa(L)   : 0.0109878 
    jd(L)   : 0.151907  
    hm(L)   : 0.00154624
    ek(L)   : 9.81910e-6
    ti(L)   : 0.00413534
    kj(L)   : 0.0368694 
    yct(L)  : 0.0885231 
    h2o(L)  : 0.219727  
    wo(fl)  : 0.000103865
    sl(fl)  : 5.87807e-5
    fo(fl)  : 1.41812e-8
    fa(fl)  : 3.11791e-8
    jd(fl)  : 0.000862050
    hm(fl)  : 0.000159068
    ek(fl)  : 3.29610e-7
    ti(fl)  : 0.000162396
    kj(fl)  : 0.00562408
    h2o(fl) : 0.985936  
    ca(plc) : 0.6       
    k(plc)  : 0.0116338 
    x(ol)   : 0.102434  
    c(ol)

In [11]:
# Now, we will simulate fractional crystallisation, 
# on each step removing all of the crystalline phases 
# that are present in the assemblage.

removal_fraction = {}
for ph in crystphases:
    removal_fraction[ph] = 1

# create outputs directory
basename = "ecrg-frac-xtn-output"
os.makedirs(basename, exist_ok=True)

modes = []
step = 0
for P,T in PT_steps:
    
    # set required windows
    context.script["calcP"] = "{} {}".format(P,P)
    context.script["calcT"] = "{} {}".format(T,T)
    
    results_filename = os.path.join(basename,"results_step_{}.txt".format(step))
    # check if results file exists
    if (not os.path.isfile(results_filename)) or rerun_all:
        # if results don't exist, execute.
        results = context.execute(timeout=600)
        # save results.
        results.save(results_filename)
        # let's save the input script too. 
        context.script.save(os.path.join(basename,"script_step_{}.txt".format(step)))
    else:
        # if the file exists, load from disk instead.
        # first create empty results object, then load.
        results = tc.Results()
        results.load(filename=results_filename)

    modes.append((results.modes,step))
    
    # set resultant rbi back on script
    print("Executing for P,T = {},{}".format(P,T))
    context.script["rbi"] = results.rbi
    # remove required phases
    for phase, removal_frac in removal_fraction.items():
        if phase in context.script["rbi"].keys():
            phase_post_solve = context.script["rbi"][phase]["mode"] 
            context.script["rbi"][phase]["mode"] *= (1.-removal_frac)
            print("Phase {} before = {} after = {}".format(phase, phase_post_solve, context.script["rbi"][phase]["mode"] ))
    
    # record new values to new dict for viz purposes
    final_modes = {}
    for key,val in context.script["rbi"].items():
        final_modes[key] = val["mode"]
    modes.append((final_modes,step))
    step += 1

Executing for P,T = 4.0,1000
Phase ksp before = 0.257009 after = 0.0
Phase q before = 0.050473 after = 0.0
Executing for P,T = 4.0,900
Phase spn before = 0.001287 after = 0.0
Phase ksp before = 0.44996 after = 0.0
Phase ilm before = 0.002231 after = 0.0
Phase q before = 0.314081 after = 0.0
Executing for P,T = 4.0,800
Phase ksp before = 0.221925 after = 0.0
Phase ilm before = 0.003119 after = 0.0
Phase opx before = 0.037208 after = 0.0
Phase q before = 0.224852 after = 0.0
Phase plc before = 0.082258 after = 0.0
Executing for P,T = 4.0,700
Phase ksp before = 0.136705 after = 0.0
Phase ilm before = 0.000752 after = 0.0
Phase bi before = 0.051837 after = 0.0
Phase q before = 0.200401 after = 0.0
Phase plc before = 0.082663 after = 0.0


In [12]:
results.phases

'fl plc ksp g opx ilm q or [liq,bi,cpx,spn] #1431'